In [2]:
import itertools
import pandas as pd
from collections import defaultdict
from itertools import islice, chain
import re
from datetime import datetime
from datetime import timedelta
import networkx as nx
import numpy as np
import os

# dataloader for new data

In [2]:
kg18 = pd.read_csv('2018.csv')
kg19_1 = pd.read_csv("2019_1.csv")
kg19_2 = pd.read_csv("2019_2.csv")
kg_19_3 = pd.read_csv("2019_3.csv")
kg_19_4 = pd.read_csv("2019_4.csv")
# data2 = pd.read_csv('new_company_relation.csv',sep='\t')
kg = pd.concat([kg18,kg19_1,kg19_2,kg_19_3,kg_19_4],axis=0,ignore_index=True)
# # order = ['head','tail','relation','sentence','time','head_code','tail_code']
# # kg_df = pd.DataFrame(all_data,columns=order)
# kg_df = all_data.drop(['Unnamed: 0','head','tail','sentence'],axis =1)
# # kg_df
# kg_df
kg = kg.sort_values(['time'])
kg

,head_code,tail_code,relation,time
385,600909.SH,601375.SH,rise,2018/10/25 11:23
703,000584.SZ,002771.SZ,supply,2018/10/25 12:31
392,002172.SZ,002432.SZ,rise,2018/10/25 13:54
7,300414.SZ,300657.SZ,be_increased_holding,2018/10/25 19:39
391,601990.SH,600369.SH,rise,2018/10/25 20:54
480,000001.SZ,600928.SH,same_industry,2018/10/25 23:17
73,000901.SZ,600118.SH,cooperate,2018/10/25 7:54
591,600880.SH,603986.SH,same_industry,2018/10/25 9:27
328,601398.SH,601288.SH,rise,2018/10/26 14:56
432,600104.SH,601238.SH,same_industry,2018/10/26 16:07


In [3]:
kg = pd.read_csv('csi300.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'csi300.csv'

In [3]:
dykg = pd.read_csv('dytuple.csv')
static_kg = pd.read_csv('SW_csi_22.csv')
static_kg['time'] = min(dykg['time'])

In [12]:
column_order = [ 'e1', 'e2','r','time']
whole_kg =  pd.concat([dykg[column_order], static_kg[column_order]], ignore_index=True)

In [15]:
static_kg

,e1,e2,r,time
0,SZ300014,SZ000100,invest,2018-01-01 00:00:00
1,SZ000100,SZ300014,invest,2018-01-01 00:00:00
2,SZ000100,SZ300014,be_reduced_holding,2018-01-02 00:00:00
3,SH601600,SH600011,reduce_holding,2018-01-02 00:00:00
4,SZ002146,SZ000786,rise,2018-01-02 00:00:00
...,...,...,...,...
33119,SZ000750,SH601375,SW_industry,2018-01-01 00:00:00
33120,SZ000750,SH600909,SW_industry,2018-01-01 00:00:00
33121,SZ000750,SH600109,SW_industry,2018-01-01 00:00:00
33122,SZ000750,SZ002500,SW_industry,2018-01-01 00:00:00


In [14]:
whole_kg['r'].value_counts()

SW_industry             20194
same_industry            4188
rise                     2468
cooperate                2156
fall                     1314
increase_holding          866
supply                    630
superior                  248
dispute                   242
invest                    220
be_increased_holding      200
reduce_holding            192
be_reduced_holding        142
compete                    28
be_invested                28
be_supplied                 8
Name: r, dtype: int64

In [8]:
relation_ls

array(['invest', 'be_reduced_holding', 'reduce_holding', 'rise',
       'same_industry', 'cooperate', 'fall', 'increase_holding',
       'dispute', 'supply', 'compete', 'be_supplied', 'superior',
       'be_increased_holding', 'be_invested'], dtype=object)

In [16]:

whole_kg = whole_kg.sort_values(by = 'time')
whole_kg = whole_kg.reset_index(drop=True)
whole_kg

,e1,e2,r,time
0,SZ300014,SZ000100,invest,2018-01-01 00:00:00
1,SZ000807,SZ000969,SW_industry,2018-01-01 00:00:00
2,SZ000807,SZ000970,SW_industry,2018-01-01 00:00:00
3,SH600808,SH601969,SW_industry,2018-01-01 00:00:00
4,SH600808,SZ000629,SW_industry,2018-01-01 00:00:00
...,...,...,...,...
33119,SZ002594,SH601238,same_industry,2023-02-06 10:29:00
33120,SH600029,SH601021,same_industry,2023-02-06 23:56:00
33121,SH601021,SH600029,same_industry,2023-02-06 23:56:00
33122,SZ000596,SZ000858,same_industry,2023-02-07 09:02:00


## timestamp

In [113]:
import dateutil.parser

def getDateTimeFromISO8601String(s):
    d = dateutil.parser.parse(s)
    return d

In [114]:
def build_links(kg_df):    
    links = []
    ts = []
    
    for i in range(len(kg_df)):
        timesting = kg_df['time'][i]
        timestamp = getDateTimeFromISO8601String(timesting)
        ts.append(timestamp)
        links.append((kg_df['e1'][i],kg_df['e2'][i],timestamp))
    return links,ts

# build subgraph

In [115]:
def subgraph(links,ts,SLICE_MONTHS,START_DATE,MAX_DATE):
    slices_links = defaultdict(lambda : nx.MultiGraph())
    slices_features = defaultdict(lambda : {})

    print ("Start date", START_DATE)
    slice_id = -1
    time_slides ={}
    # Split the set of links in order by slices to create the graphs.
    for (a, b, time) in links:
        prev_slice_id = slice_id
        
        datetime_object = time
        if datetime_object > MAX_DATE:
            months_diff = (MAX_DATE - START_DATE).days//30  # 
            
        else:
            months_diff = (datetime_object - START_DATE).days//30

        slice_id = months_diff // SLICE_MONTHS      
        slice_id = max(slice_id, 0)
        if str(slice_id) not in time_slides:
            time_slides[str(slice_id)] = []
        time_slides[str(slice_id)].append(time)

        if slice_id == 1+prev_slice_id and slice_id ==0:
            slices_links[slice_id] = nx.MultiGraph()
            
        if slice_id == 1+prev_slice_id and slice_id > 0:
            slices_links[slice_id] = nx.MultiGraph()
            slices_links[slice_id].add_nodes_from(slices_links[slice_id-1].nodes(data=True))
            slices_links[slice_id].add_edges_from(slices_links[slice_id-1].edges(data=True)) 
            ## ***** keep the edges from the last graph
            
        if a not in slices_links[slice_id]:
            slices_links[slice_id].add_node(a)
        if b not in slices_links[slice_id]:
            slices_links[slice_id].add_node(b)
        slices_links[slice_id].add_edge(a,b, date=datetime_object)
        


    for slice_id in slices_links:
        print ("# nodes in slice", slice_id, len(slices_links[slice_id].nodes()))
        print ("# edges in slice", slice_id, len(slices_links[slice_id].edges()))
        temp = np.identity(len(slices_links[max(slices_links.keys())].nodes()))
        slices_features[slice_id] = {}
        for idx, node in enumerate(slices_links[slice_id].nodes()):
            slices_features[slice_id][node] = temp[idx]

    return slices_features,slices_links,time_slides

In [116]:
links,ts = build_links(whole_kg)
min(ts), max(ts)

(datetime.datetime(2018, 1, 1, 0, 0), datetime.datetime(2023, 2, 7, 9, 2))

In [117]:
SLICE_WEEKS,START_DATE,MAX_DATE = 1, min(ts), max(ts)
slices_features,slices_links,time_slides = subgraph(links,ts,SLICE_WEEKS,START_DATE,MAX_DATE)

Start date 2018-01-01 00:00:00
# nodes in slice 0 680
# edges in slice 0 20390
# nodes in slice 1 682
# edges in slice 1 20534
# nodes in slice 2 685
# edges in slice 2 20710
# nodes in slice 3 688
# edges in slice 3 20892
# nodes in slice 4 695
# edges in slice 4 21086
# nodes in slice 5 697
# edges in slice 5 21256
# nodes in slice 6 698
# edges in slice 6 21382
# nodes in slice 7 700
# edges in slice 7 21572
# nodes in slice 8 702
# edges in slice 8 21736
# nodes in slice 9 703
# edges in slice 9 21880
# nodes in slice 10 706
# edges in slice 10 22278
# nodes in slice 11 708
# edges in slice 11 22686
# nodes in slice 12 710
# edges in slice 12 22886
# nodes in slice 13 710
# edges in slice 13 22988
# nodes in slice 14 713
# edges in slice 14 23240
# nodes in slice 15 715
# edges in slice 15 23482
# nodes in slice 16 716
# edges in slice 16 23640
# nodes in slice 17 717
# edges in slice 17 23874
# nodes in slice 18 718
# edges in slice 18 24155
# nodes in slice 19 719
# edges in slic

In [65]:

grouped_data = {}
cumulative_data = pd.DataFrame()
time_index = []
whole_kg['time'] = pd.to_datetime(whole_kg['time'])  
whole_kg = whole_kg.sort_values('time')
for i, (name, group) in enumerate(whole_kg.groupby(pd.Grouper(key='time', freq='M')), start=0):
    cumulative_data = pd.concat([cumulative_data, group])
    grouped_data[i] = cumulative_data
    time_index.append([min(cumulative_data['time']),max(cumulative_data['time'])])

time_index_df = pd.DataFrame(time_index,columns=['start','end'])
# time_index_df.to_csv('output/newcsi/time_index.csv')


In [66]:
len(time_index)

62

In [ ]:
slices_links = defaultdict(lambda : nx.MultiGraph())
slices_features = defaultdict(lambda : {})
for i in range(len(grouped_data)):
    combined_column = pd.concat([grouped_data[i]['e1'], grouped_data[i]['e2']])
    unique_nodes = combined_column.drop_duplicates()
    nodes_list = unique_nodes.values.tolist()
    
    edges = [(row['e1'], row['e2'], {'date': row['time']}) for _, row in grouped_data[i].iterrows()]
    slices_links[i].add_nodes_from(nodes_list)
    slices_links[i].add_edges_from(edges)
#     print(len(grouped_data[i]))
#     for j in range(len(grouped_data[i])):
#         print(j)
#         print(grouped_data[i]['e2'][j])
# #         slices_links[i].add_edge(grouped_data[i]['e1'][j],grouped_data[i]['e2'][j],date=grouped_data[i]['time'][j])
    
# for slice_id in slices_links:
#     print ("# nodes in slice", slice_id, len(slices_links[slice_id].nodes()))
#     print ("# edges in slice", slice_id, len(slices_links[slice_id].edges()))

In [ ]:
for slice_id in slices_links:
    print ("# nodes in slice", slice_id, len(slices_links[slice_id].nodes()))
    print ("# edges in slice", slice_id, len(slices_links[slice_id].edges()))
    temp = np.identity(len(slices_links[max(slices_links.keys())].nodes()))

    slices_features[slice_id] = {}
    # print(slices_links)
    for idx, node in enumerate(slices_links[slice_id].nodes()):
        # print(idx,node)
        slices_features[slice_id][node] = temp[idx]

In [ ]:

for i in time_slides.keys():
    print("slice:",i)
    print(min(time_slides[i]),max(time_slides[i]))


silde 1: start from 3.1


# remap

In [118]:
from scipy.sparse import csr_matrix

def remap(slices_graph, slices_features):
    all_nodes = []
    for slice_id in slices_graph:
        assert len(slices_graph[slice_id].nodes()) == len(slices_features[slice_id])
        all_nodes.extend(slices_graph[slice_id].nodes())
    all_nodes = list(set(all_nodes))
    #print(slices_graph[1].nodes())
    print ("Total # nodes", len(all_nodes), "max idx", max(all_nodes))
    # print ("Total # nodes", len(all_nodes))
    ctr = 0
    node_idx = {}
    idx_node = []
    for slice_id in slices_graph:
        for node in slices_graph[slice_id].nodes():
            if node not in node_idx:
                node_idx[node] = ctr
                idx_node.append(node)
                ctr += 1
    slices_graph_remap = []
    slices_features_remap = []
    for slice_id in slices_graph:
        G = nx.MultiGraph()
        for x in slices_graph[slice_id].nodes():
            G.add_node(node_idx[x])
        for x in slices_graph[slice_id].edges(data=True):
            G.add_edge(node_idx[x[0]], node_idx[x[1]], date=x[2]['date'])
        assert (len(G.nodes()) == len(slices_graph[slice_id].nodes()))
        assert (len(G.edges()) == len(slices_graph[slice_id].edges()))
        slices_graph_remap.append(G)
    
    for slice_id in slices_features:
        features_remap = []
        print(len(slices_graph_remap[slice_id].nodes()))
        for x in slices_graph_remap[slice_id].nodes():
            # print(slices_features[slice_id][idx_node[x]])
            features_remap.append(slices_features[slice_id][idx_node[x]])
            # features_remap.append(np.array(slices_features[slice_id][idx_node[x]]).flatten())
        features_remap = csr_matrix(np.squeeze(np.array(features_remap)))
        slices_features_remap.append(features_remap)
    return slices_graph_remap, slices_features_remap, idx_node,node_idx

In [119]:
slices_links_remap, slices_features_remap, idx_node,node_idx = remap(slices_links, slices_features)

Total # nodes 743 max idx SZ300999
680
682
685
688
695
697
698
700
702
703
706
708
710
710
713
715
716
717
718
719
720
720
720
720
720
722
723
725
726
728
729
729
729
733
733
734
734
735
736
736
738
740
740
740
740
741
741
741
741
742
742
742
743
743
743
743
743
743
743
743
743
743
743


In [18]:
idx_node

{'SH600010': 0,
 'SH600169': 1,
 'SZ300104': 2,
 'SH600104': 3,
 'SH601066': 4,
 'SZ300498': 5,
 'SH601099': 6,
 'SZ300059': 7,
 'SH601988': 8,
 'SH601328': 9,
 'SH600900': 10,
 'SH601398': 11,
 'SZ300142': 12,
 'SZ300122': 13,
 'SH600079': 14,
 'SH601162': 15,
 'SH600029': 16,
 'SH600115': 17,
 'SH600011': 18,
 'SH600875': 19,
 'SH600570': 20,
 'SH601519': 21,
 'SH601229': 22,
 'SH600655': 23,
 'SH601939': 24,
 'SH600880': 25,
 'SH603986': 26,
 'SH601375': 27,
 'SH600909': 28,
 'SH601788': 29,
 'SH600143': 30,
 'SH601288': 31,
 'SH601238': 32,
 'SH601211': 33,
 'SH600036': 34,
 'SH601111': 35,
 'SH601318': 36,
 'SH600519': 37,
 'SH600066': 38,
 'SH600588': 39,
 'SH601108': 40,
 'SH600999': 41,
 'SH601958': 42,
 'SH603993': 43,
 'SZ300413': 44,
 'SH600369': 45,
 'SH600600': 46,
 'SH601688': 47,
 'SH600009': 48,
 'SH600015': 49,
 'SH600276': 50,
 'SH601966': 51,
 'SH600958': 52,
 'SH600887': 53,
 'SH601555': 54,
 'SH601319': 55,
 'SH601628': 56,
 'SH600030': 57,
 'SZ300017': 58,
 'SH600

# overall


### generate the input for temporal random walk

In [ ]:
for slice_id in slices_links:
    for a,b,t in slices_links[slice_id].edges(data=True):
        print(a,b,t['date'])
        n1 = node_idx[a]
        n2 = node_idx[b]
        time = int(t['date'].timestamp())

In [121]:
np.savez('graphs_123.npz', graph=slices_links_remap)
np.savez('features_123.npz', feats=slices_features_remap)

In [21]:
import json
json_str = json.dumps(node_idx)
with open('company2id.json', 'w') as json_file:
    json_file.write(json_str)

# 备用

In [15]:
with open('supply_graphs.npz', 'wb') as f:
    np.savez(f, graph=slices_links_remap)

In [ ]:
# to id 
company_ls_1 = list(kg_df['head'])
company_ls_2 = list(kg_df['tail'])
company_ls = company_ls_1 + company_ls_2

In [ ]:
company_ls = list(set(company_ls))

In [ ]:
company_dict = dict(company_ls)

ValueError: dictionary update sequence element #0 has length 9; 2 is required

In [ ]:
company_ls


NameError: name 'company_ls' is not defined